**Fruit Classification**: An exercise involving transfer learning

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
dataset_path = 'gdrive/My Drive/Deep Learning/Fruit Classification'

Mounted at /content/gdrive


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
base_dir = dataset_path + "/data/test"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

In [ ]:
image_size = 128 # All images will be resized to 224 x 224
batch_size = 32

**Keras' Image Datagenerator**: Performs data augmentation accoring to the parameters given, which can be determined based on the results obtained

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,rotation_range=30,horizontal_flip=True,vertical_flip = True,validation_split =0.1)
#train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
import glob
data = glob.glob('gdrive/My Drive/Deep Learning/*')
print(data)

['gdrive/My Drive/Deep Learning/MNIST with CNN', 'gdrive/My Drive/Deep Learning/Char Prediction', 'gdrive/My Drive/Deep Learning/sentiment analysis', 'gdrive/My Drive/Deep Learning/Pneumothorax', 'gdrive/My Drive/Deep Learning/Fruit Classification']


The next two cells generate the train and validation images

In [ ]:
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                batch_size=batch_size,
                target_size=(image_size, image_size),
                class_mode='categorical',
                subset = 'training')

Found 10802 images belonging to 6 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
                train_dir, # Source directory for the validation images
                batch_size=batch_size,
                target_size=(image_size, image_size),
                class_mode='categorical',
                subset = 'validation')

Found 1198 images belonging to 6 classes.


VGG16 used for transfer learning. Using Imagenet weights


In [ ]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

In [ ]:
base_model.trainable = False

To the base model, a dense layer of 1024 neurons is added, along with drop out. Finally there is the output layer of 6 neurons

In [ ]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(6, activation='softmax')
])
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
'''model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (image_size,image_size,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(6, activation = "softmax"))'''

'model = Sequential()\n\nmodel.add(Conv2D(filters = 32, kernel_size = (5,5),padding = \'Same\', \n                 activation =\'relu\', input_shape = (image_size,image_size,3)))\nmodel.add(Conv2D(filters = 32, kernel_size = (5,5),padding = \'Same\', \n                 activation =\'relu\'))\nmodel.add(MaxPool2D(pool_size=(2,2)))\nmodel.add(Dropout(0.25))\n\n\nmodel.add(Conv2D(filters = 64, kernel_size = (3,3),padding = \'Same\', \n                 activation =\'relu\'))\nmodel.add(Conv2D(filters = 64, kernel_size = (3,3),padding = \'Same\', \n                 activation =\'relu\'))\nmodel.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))\nmodel.add(Dropout(0.25))\n\nmodel.add(Conv2D(filters = 128, kernel_size = (3,3),padding = \'Same\', \n                 activation =\'relu\'))\nmodel.add(Conv2D(filters = 128, kernel_size = (3,3),padding = \'Same\', \n                 activation =\'relu\'))\nmodel.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))\nmodel.add(Dropout(0.25))\n\nmodel.add(Conv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0,amsgrad=False),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
epochs = 50

steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

filepath= dataset_path + "/weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers = 4,
                              shuffle = True,
                              callbacks=callbacks_list,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

Epoch 1/50
336/337 [============================>.] - ETA: 0s - loss: 0.2902 - acc: 0.8964
Epoch 00001: val_loss improved from inf to 0.36966, saving model to gdrive/My Drive/Deep Learning/Fruit Classification/weights-improvement-01-0.87.h5
337/337 [==============================] - 43s 128ms/step - loss: 0.2908 - acc: 0.8962 - val_loss: 0.3697 - val_acc: 0.8674
Epoch 2/50
336/337 [============================>.] - ETA: 0s - loss: 0.2505 - acc: 0.9099
Epoch 00002: val_loss improved from 0.36966 to 0.30688, saving model to gdrive/My Drive/Deep Learning/Fruit Classification/weights-improvement-02-0.88.h5
337/337 [==============================] - 43s 129ms/step - loss: 0.2507 - acc: 0.9098 - val_loss: 0.3069 - val_acc: 0.8775
Epoch 3/50
336/337 [============================>.] - ETA: 0s - loss: 0.2365 - acc: 0.9138
Epoch 00003: val_loss improved from 0.30688 to 0.25860, saving model to gdrive/My Drive/Deep Learning/Fruit Classification/weights-improvement-03-0.90.h5
337/337 [============

In [ ]:
model.save('mymodel.h5')

In [ ]:
#from tf.keras.models import load_model
import tensorflow
model = tensorflow.keras.models.load_model(dataset_path+'/weights-improvement-36-0.97.h5')

W0824 19:01:12.590719 140212026992512 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0824 19:01:12.592272 140212026992512 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
test_dir = validation_dir

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(test_dir, target_size = (image_size,image_size),batch_size=32,shuffle=False)

Found 18000 images belonging to 1 classes.


In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

pred=model.predict_generator(test_generator,verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)

labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]

 57/563 [==>...........................] - ETA: 55s

KeyboardInterrupt: ignored

In [ ]:
files = [file.split('/')[-1] for file in test_generator.filenames]

In [ ]:
import pandas as pd

In [ ]:
result = pd.DataFrame({"file_id":files,"prediction":predictions})

In [ ]:
result.to_csv(dataset_path+"output_vgg_96.csv",header=True)